In [ ]:
import sklearn
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import re
import seaborn as sns
import pyfpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from sklearn.model_selection import train_test_split


class FpGrowth:
     
    def __init__(self, min_support, test_size, test_or_train):
        #minimum support akan digunakan untuk menyaring data
        self.min_supp = min_support
        self.ts = test_size
        self.tot = test_or_train
        
        
    #fungsi menghapus data yang memiliki value atau nilai yang null
    def data_cleaning(dataset):
        data_cleaning = dataset.dropna()
        
        return data_cleaning
    
    
    #fungsi ini berfungsi untuk menghapus line yang digunakan pada data untuk mempedakan produk yang dibeli
      #fungsi mengubah karakter "|" menjadi komma (,) dan menampungnya di atribut baru yaitu 'changes_line'
    def delete_line(dataset_line):
        dataset_line['products'] = dataset_line['products'].apply(lambda x: 
                                                    [re.sub(r"[\|\&]", ", ", word) for word in x.split(',')])
        
        data = dataset_line["products"].tolist()
        return data
        
    
    #fungsi ini digunakan untuk membuat data menjadi array dua dimensi
    def StrArray(itemset):
        index = []
        coba = []
        idx = []
        items = []
        sub = []
    
        #mengubah baris menjadi array dalam array
        for arr in itemset:
            for row in arr:
                coba = row.split(',')
                index.append(coba)
            
        #menghapus Whitespaces dari String menggunakan Regex   
        for arr in index:
            for row in range(len(arr)):
                idx = re.sub(r"^\s+|\s+$", "", arr[row])
                arr[row] = idx
            items.append(arr)
        
        #menghapus item yang duplikat dalam satu satu itemset
        for arr in items:
            for row in arr:
                if arr.count(row) > 1 :
                    arr.remove(row)    
            sub.append(arr)
        
        return sub
    
    #fungsi untuk mengahasilkan gambar tree berdasarkan produk 
    def createTree(self):
        data = FpGrowth.load_data(self.tot, self.min_supp)
        dataSet = FpGrowth.createInitSet(data)
        headerTable = {}
        for trans in dataSet:
            for item in trans:
                headerTable[item] =headerTable.get(item, 0) + dataSet[trans]
        for k in list(headerTable):
            if headerTable[k] < self.min_supp:
                del (headerTable[k])
        freqItemSet = set(list(headerTable))
        if len(freqItemSet) == 0:
            return None, None
        for k in headerTable:
            headerTable[k] = [headerTable[k], None]
        retTree = treeNode('Null Set', 1, None)
        for tranSet, count in dataSet.items():
            localD = {}
            for item in tranSet:
                if item in freqItemSet:
                    localD[item] = headerTable[item][0]
            if len(localD) > 0:
                orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p:p[1], reverse=True)]
                FpGrowth.updateTree(orderedItems, retTree, headerTable, count)
        return retTree, headerTable
    
    def createInitSet(dataSet):
        retDict = {}
        for trans in dataSet:
            retDict[frozenset(trans)] = 1
        return retDict
    
    
    def updateTree(items, inTree, headerTable, count):
        if items[0] in inTree.children:
            inTree.children[items[0]].inc(count)
        else:
            inTree.children[items[0]] = treeNode(items[0], count, inTree)
            if headerTable[items[0]][1] == None:
                headerTable[items[0]][1] = inTree.children[items[0]]
            else: 
                FpGrowth.updateHeader(headerTable[items[0]][1],
                            inTree.children[items[0]])
            
        if len(items) > 1:
            FpGrowth.updateTree(items[1::], inTree.children[items[0]], headerTable, count)
     
    
        
    def ascendTree(leafNode, prefixPath):
        if leafNode.parent != None:
            prefixPath.append(leafNode.name)
            ascendTree(leafNode.parent, prefixPath)
    
            
    def findPrefixPath(basePat, treeNode):
        condPats = {}
        while treeNode != None:
            prefixPath = []
            ascendTree(treeNode, prefixPath)
            if len(prefixPath) > 1:
                condPats[frozenset(prefixPath[1:])] = treeNode.count
            treeNode = treeNode.nodeLink
        return condPats
        
    
    def updateHeader(nodeToTest, targetNode):
        while (nodeToTest.nodeLink != None):
            nodeToTest = nodeToTest.nodeLink
        nodeToTest.nodeLink = targetNode
    
    
    
    def load_data(tt ,n_split):  
    
        #mengambil data
        data = pd.read_csv("F:/Catatan Materi/Semester 7/DAMI_PD/Projek/Dataset/FIX2/FIX.csv")
    
        #menghapus baris yang memiliki nilai null atau NAN
        data1 = FpGrowth.data_cleaning(data)
   
        #mengubah karakter "|" menjadi komma (,) dan menampungnya di atribut baru yaitu 'changes_line'
        dl_line = FpGrowth.delete_line(data1)
    
        #mengambil kolom changes_line
        
        data3=  FpGrowth.StrArray(dl_line)
        train, test = train_test_split(data3, test_size=n_split)
        
        if tt=='test':
            data4 = test
        else:
            data4 = train
        #mengubah row atau baris menjadi array dalam array
   
    
        """simpDat = [['jojo milk asli susu kambing', 'teh sosor selerah ayam', 'fresker alay baba hoho'],
                ["teh sosor  selerah ayam", "masker kulit kerbau", "betadin"],
                ["jojo milk asli susu kambing", "masker kulit kerbau", "fresker alay baba hoho"],
                ["teh sosor selerah ayam", "sari roti"],
                ["bear", "jojo milk asli susu kambing", "teh sosor selerah ayam"],
                ["bear", "jojo milk asli susu kambing", "teh sosor selerah ayam", "fresker alay baba hoho"]]"""
        
        
        return data4
    
    def test(self):
        tt = self.tot
        tr = self.ts
        dataset =  FpGrowth.load_data(tt,tr)
        te = TransactionEncoder()
        te_ary = te.fit(dataset).transform(dataset)
        df = pd.DataFrame(te_ary, columns=te.columns_)
        data = fpgrowth(df, self.min_supp, use_colnames=True)
        md1 = pyfpgrowth.find_frequent_patterns(dataset, 100)
        md2 = pyfpgrowth.generate_association_rules(md1, self.min_supp)
        return data, md2
    
    
    def train(self):
        tt = self.tot
        tr = self.ts
        dataset =  FpGrowth.load_data(tt,tr)
        te = TransactionEncoder()
        te_ary = te.fit(dataset).transform(dataset)
        df = pd.DataFrame(te_ary, columns=te.columns_)
        data = fpgrowth(df, self.min_supp, use_colnames=True)
        #menghitung barang yang muncul minimal 100 kali
        md1 = pyfpgrowth.find_frequent_patterns(dataset, 100)
        #memprediksikan nilai kemunculan barang 
        md2 = pyfpgrowth.generate_association_rules(md1, self.min_supp)
        return data, md2
    
    
class treeNode: 
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children= {}
    
    def inc(self, numOccur):
        self.count += numOccur
    
    #menampilkan tree jika diperlukan
    def disp(self, ind=1):
        print (' '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)
            


In [2]:
test = FpGrowth(0.01, 0.4, 'test')
data, prediksi = test.test()

In [ ]:
data.head()

In [ ]:
print(prediksi)

In [4]:
tree, header=test.createTree()

In [5]:
tree.disp()

  Null Set   1
   Clementines   10
    Pineapple Yogurt 2%   1
     Total with Mango Greek Strained Yogurt   1
      Original Fat Free Liquid Creamer   1
    Advanced Hand Sanitizer Bonus Sized Pump Aloe   1
    Chocolate Hazelnut Spread   1
     Regular Roll   1
      Mountain Breeze Laundry Detergent   1
    Frozen Organic Blueberries   1
     Aged White Cheddar Popcorn   1
      Organic White Bread   1
    Chicken   1
     Maple Breakfast Sausage   1
      Heavy Whipping Cream   1
       Original Baked Whole Grain Wheat Crackers   1
        Appley Ever After Organic Juice Drink   1
         Premium Unsweetened Iced Tea   1
          Mix Popcorn   1
           Annie's Bunny Fruit Snacks Variety   1
    Organic Unsweetened Vanilla Almond Milk   1
     Pink Lady Apples   1
      with Crispy Almonds Cereal   1
    Philadelphia Original Cream Cheese   1
     Lavender Dishwashing Liquid With Olay   1
      Mandarin Orange in 100% Fruit Juice   1
       Parmesan Cheese   1
        Lavender

      100 Calorie  Per Bag Popcorn   1
       Organic Whole Grassmilk Milk   1
        Organic Cripps Pink Apples   1
         Curate Cherry Lime Sparkling Water   1
          Crackers Almond Flour Farmhouse Sharp Cheddar   1
           White Cheddar Snack Crackers Cheddar Bunnies   1
            Organic Himalayan Pink Popcorn   1
             Carob Chip   1
              Essential Greens With Lime  Vegetable and Fruit Juice Blend   1
               Sweet Greens and Lemon Vegetable and Fruit Juice Blend   1
      Asparagus   1
       Organic Gala Apples   1
        Organic Grade A Free Range Large Brown Eggs   1
         Organic Whole String Cheese   1
          Strawberry   1
           Organic Smoked Turkey Breast   1
            Natural Chicken   1
             Sage Breakfast Sausage   1
              Broccoli Florets   1
               Pad Thai   1
                YoKids Squeeze! Organic Strawberry Flavor Yogurt   1
                 Organic Yukon Select Fries   1
                  

          Organic Navel Orange   1
           Hass Avocados   1
            Organic Garbanzo Beans   1
             Organic Tahini   1
              Organic Bunch Beets   1
               Total 2% All Natural Low Fat 2% Milkfat Greek Strained Yogurt   1
                Reduced Fat Milk   1
                 Golden Flaxseed   1
                  Lemons   1
      Carrots   1
       Organic Gala Apples   1
        Organic Large Brown Grade AA Cage Free Eggs   1
         Strawberry/Vanilla Yogurt   1
          Organic 1% Low Fat Milk   1
           YoKids Blueberry   1
            Tilapia Filet   1
             Organic Red Radish   1
              Yobaby Organic Plain Yogurt   1
               Organic  Whole Milk   1
                Fat Free Plain Yogurt   1
                 California Black Bread   1
     Coconut Water   1
      Distilled Water   1
       Butter Lettuce   1
     Half   1
      Honeycrisp Apple   1
       Organic Tomato Cluster   1
        Organic Half   1
         Total 2%

        Lemon Yogurt   1
     Organic Large Extra Fancy Fuji Apple   1
      Authentic French Brioche   1
       Country Stand Juice   1
        Farmhouse Culture Classic Caraway Kraut   1
     Organic Granny Smith Apple   1
      All Natural Lemonade   1
    Organic Lemon   2
     Organic Navel Orange   1
      Organic Snipped Green Beans   1
       Organic Fresh Carrot Chips   1
     Red Peppers   1
      Organic Italian Parsley Bunch   1
       Orange Bell Pepper   1
        Roasted Red Pepper Hummus   1
         Organic Coconut Milk   1
          Yellow Bell Pepper   1
           Tommy/Kent/Keitt/Haden Mango   1
            Organic Extra Large Brown Eggs   1
             Purity Farms Ghee Clarified Butter   1
              Mandarin Oranges   1
               Lavender Hand Soap   1
    Unsweetened Vanilla Almond Milk   2
     Organic Bread with 21 Whole Grains   1
      Sea Salt Roasted Seaweed   1
       Blueberry Protein Bar   1
        Black Cherry   1
     Lemon   1
      Rich  

    Seedless Red Grapes   1
     Garlic   1
      Organic Tomato Cluster   1
       Sparkling Water Grapefruit   1
        Organic Reduced Fat Milk   1
         Organic Large Grade AA Brown Eggs   1
          Organic Creamy Peanut Butter   1
           Frozen Organic Wild Blueberries   1
            Celery Hearts   1
             Natural Sour Cream   1
              Roasted   1
               Honey Greek Yogurt   1
                All Natural Virgin Lemonade   1
                 Organic Quick Oats   1
                  Organic Unsweetened Vanilla Almond Milk   1
                   Organic Whole Wheat Bread   1
                    Organic Brussels Sprouts   1
                     Salted Almonds   1
                      Fine Herbs Gournay Cheese   1
                       Organic Whole Milk Strawberry Kefir   1
                        Mini Falafel With Tahini Sauce   1
    Bunch   1
     Frozen Broccoli Florets   1
    Total 0% Nonfat Plain Greek Yogurt   1
     California Veggie Burger

...

        Organic Cream Cheese Bar   1
         Iced Oatmeal Cookie Kid Z Bar   1
          Apple Sausage   1
           Organic California White Basmati Rice   1
    Organic Green Cabbage   1
     Sweet Onion   1
      Golden Sweet Potato   1
       Whole White Mushrooms   1
        Creamy Ranch Dressing   1
         Carrot Cake   1
          Angel Sweet Tomatoes   1
           Palmetto Cheese Original   1
    Red Vine Tomato   1
     Fat Free Milk   1
      Mini Sweet Peppers   1
       Dark Chocolate  with A Touch Of Sea Salt   1
        Special K Chocolatey Delight Cereal   1
         Mozzarella Light Low Moisture Part Skim String Cheese   1
          Ultra Strong Unscented 2-Ply Bathroom Tissue   1
    Seedless Red Grapes   1
     Organic Whole Milk   1
      Yellow Onions   1
       Broccoli Crown   1
        Organic Banana   1
         Organic Peeled Whole Baby Carrots   1
          Organic Russet Potato   1
           Lightly Salted Baked Snap Pea Crisps   1
            Organic S

             Peanut Butter Dark Chocolate Protein Chewy Bars   1
              Ibuprofen Liqui-Gels   1
               Honey BBQ Flavored Chicken Strips   1
                Antioxidant Infusions Costa Rica Clementine   1
                 Crispy Golden Spud Puppies   1
                  Fully Cooked Batter Dipped Chicken Breast Tenders   1
                   Antioxidant Infusions Ipanema Pomegranate Beverage   1
                    Big Original baked Cheese Crackers   1
   Original Cereal   1
    Gluten Free   1
     Sopressata   1
      Calabrese Spicy Salami   1
       Deli Thin-Sliced Honey Ham   1
   Plain Cultured Goat Milk Kefir   1
    Stage 1 - Just Prunes   1
     Gin Gins Original Chewy Ginger Candy   1
      Organic Sumac   1
       Elderberry Passion Sparkling Probiotic Beverage   1
        Organic Strawberry Lemonade Kombucha   1
   Premium Pure Cane Granulated Sugar   1
    Coke Zero Diet Soda   1
     Original Cooking Spray   1
      Unsalted Margarine   1
       No Stick

   Large Grade AA Organic Eggs   1
    Horchata Original Milk   1
     Rice Drink   1
      Sweet Iced Tea   1
       Heat-N-Go! Maple Burst'n Mini Pancakes   1
   Pure Organic   1
    Calm Chamomile Herbal Tea   1
     Earl Grey Tea   1
      Lemon Ginger Tea Bags   1
       Green Blueberry Slim Life Tea   1
        EveryDay Detox Tea   1
   Clean Linen   1
   6-7/8 In   1
   Blanc De Noir Champagne   1
   Bread   1
   Oven Roasted Turkey Breast   2
    Fresh Cut Baby Carrots   1
    Kitchen Cheese Enchilada   1
     Turkey   1
      Flame Grilled Chicken Breast Strips   1
       Cheddar Cracker Stackers   1
        Stimulations Ecstasy Ultrasmooth Lubricant Premium Latex Condoms   1
   Purple Carrot   1
    Organic Stage 4 Green Bean Pear   1
     Pear Baby Food   1
      Pea Baby Food   1
       Organic Stage 4 Spinach Mango   1
        Baby Food Stage 2 Blueberry Pear   1
         Blueberries Organic Pureed Baby Food   1
          Stage 1 Apples Sweet Potatoes Pumpkin   1
         

    Versatile Stain Remover   1
     Chunky Beef Meaty Ground Dog Food   1
   Organic Red Cabbage   2
    Pickled Sushi   1
    Lavender scent   1
     Blue Eucalyptus   1
   Unsalted Butter   3
    Muffins   1
     Cultured Low Fat Buttermilk   1
    Apples   1
     Organic Extra Large Brown Eggs   1
      Organic Roasted Sliced Chicken Breast   1
       Baby Food Pears Squash   1
    Organic Heavy Whipping Cream   1
   Baking Powder Aluminum-Free   1
   Total 0% Nonfat Plain Greek Yogurt   2
    Dijon Mustard   1
     Organic Raw Unfiltered Apple Cider Vinegar   1
      Blueberry Lemonade   1
       Cocoa Krispies Cereal   1
        Honey Clusters with Toasted Coconut Gluten Free   1
         Elbow Macaroni   1
          Healthy Grains Oats   1
           Yukon Gold Potato   1
            3D White Arctic Fresh Toothpaste   1
             Double Roll Bath Tissue   1
    One Ply Choose A Size Big Roll Paper Towel Rolls   1
     Cool Touch Coconut Oil   1
      Aloe Facial Tissues   1
 

    Mini Watermelon   1
     Coconut Chocolate Chip   1
      Buttermilk Pancakes   1
       Delights Turkey Sausage Breakfast Bowl   1
        Delights Turkey Sausage   1
         Mixed Berry BelVita Bites   1
          Chocolate Breakfast Bites   1
           Light Vanilla Soymilk   1
   Fat Free Milk   2
    Natural Choice Deli Oven Roasted Turkey   1
     Part-Skim Mozzarella Cheese   1
      Mozarella String Cheese   1
       Flaked Coconut Unsweetened   1
    Soft   1
     Large Grade AA Eggs   1
      Hot Dog Buns Enriched 8 Count   1
       Crispy Crowns! Seasoned Shredded Potatoes   1
        Glacier Freeze   1
         Bun Length Skinless Beef Franks   1
          Strong Bathroom Tissue 2 Ply   1
           Thirsty Ultra Big Roll towel White Vsa   1
            Smoked Beef Rope Sausage   1
             Original Jambalaya Rice Mix   1
              Santa Cruz Sourdough   1
               Major Dickason's Blend Dark Roast K-Cups   1
   Wide Egg Noodles   1
    Take   1
     Dar

In [6]:
datas, predis= test.train()

In [8]:
datas.head()

,support,itemsets
0,0.025581,(Organic Whole Milk)
1,0.099475,(Banana)
2,0.017871,(Raspberries)
3,0.018358,(Organic Baby Carrots)
4,0.012942,(Blueberries)


In [9]:
predis

{('Organic',): (('Raw Strawberry Serenity Kombucha',), 0.09074573225516622),
 ('Raw Strawberry Serenity Kombucha',): (('Organic',), 1.0),
 ('Cream Top Smooth',): (('Creamy Vanilla Yogurt',), 0.49047619047619045),
 ('Creamy Vanilla Yogurt',): (('Cream Top Smooth',), 1.0),
 ('Gluten-Free Original Crackers',): (('Organic Wheat-Free',), 1.0),
 ('Organic Wheat-Free',): (('Gluten-Free Original Crackers',),
  0.9459459459459459),
 ('Pepper Krinkle Chips',): (('Salt',), 1.0),
 ('Salt',): (('Pepper Krinkle Chips',), 0.527363184079602),
 ('Crispy Restaurant Style Tortilla Chips',): (('Organic Lightly Salted Sea Salt Thin',),
  1.0),
 ('Organic Lightly Salted Sea Salt Thin',): (('Crispy Restaurant Style Tortilla Chips',),
  1.0),
 ('Clear Dish Liquid',): (('Natural Free',), 0.9813084112149533),
 ('Natural Free',): (('Clear Dish Liquid',), 0.6907894736842105),
 ('Pear Stage 2 Baby Food',): (('Spinach Peas',), 1.0),
 ('Spinach Peas',): (('Pear Stage 2 Baby Food',), 1.0),
 ('Organic Beans',): (('Ric